Import Libraries

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
import networkx as nx
import metis

Download Cora Dataset

In [2]:
dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


Get the DGL graph

In [4]:
cora_dgl = dataset[0]

Convert to a networkx graph

In [5]:
cora_nx = dgl.to_networkx(cora_dgl)
# nx.write_gexf(G,"dgl_cora.gexf")

Get the ids of the largest connected component

In [6]:
components = nx.strongly_connected_components(cora_nx)
largest_component = max(components, key=len)

Get the ids of the disconnected nodes

In [7]:
disconnected_nodes = torch.LongTensor(list(set(cora_nx.nodes()).difference(largest_component)))

remove disconnected nodes from DGL graph

In [8]:
cora_dgl.remove_nodes(disconnected_nodes)
cora_dgl.num_nodes()

2485

Save the largest connected component

In [9]:
cora_c_nx = dgl.to_networkx(cora_dgl)
nx.write_gexf(cora_c_nx,"cora_connected.gexf")

In [10]:
print('Node features')
print(cora_dgl.ndata)
print('Edge features')
print(cora_dgl.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 0, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


METIS Partitioning

In [16]:
_, parts = metis.part_graph(cora_c_nx,2)

Create a dictionary that maps node ids to partition ids

In [17]:
part_dict = {}
for id,part_id in enumerate(parts):
    part_dict[id] = part_id

Label the nodes with their partition id and save to visualize in Gephi

In [18]:
nx.set_node_attributes(cora_c_nx,part_dict,name="partition")
nx.write_gexf(cora_c_nx,"dgl_cora_connected_parts.gexf")

Get node ids for each partition

In [30]:
parts_tensor = torch.Tensor(parts)
sg0_nodes = (parts_tensor == 0).nonzero()[:,0].tolist()
sg1_nodes = (parts_tensor == 1).nonzero()[:,0].tolist()

Create DGL subgraphs using partition list. This splits all the node features as well which is very conveniant

In [32]:
sg0 = dgl.node_subgraph(cora_dgl, sg0_nodes)
sg1 = dgl.node_subgraph(cora_dgl, sg1_nodes)

In [39]:
print("sg0 test:",sum(sg0.ndata['test_mask']==True))
print("sg0 val:",sum(sg0.ndata['val_mask']==True))
print("sg0 train:",sum(sg0.ndata['train_mask']==True))
print("sg1 test:",sum(sg1.ndata['test_mask']==True))
print("sg1 val:",sum(sg1.ndata['val_mask']==True))
print("sg1 train:",sum(sg1.ndata['train_mask']==True))

sg0 test: tensor(437)
sg0 val: tensor(213)
sg0 train: tensor(64)
sg1 test: tensor(478)
sg1 val: tensor(246)
sg1 train: tensor(58)


In [40]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [41]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [45]:
model = GCN(cora_dgl.ndata['feat'].shape[1], 16, dataset.num_classes)
model0 = GCN(sg0.ndata['feat'].shape[1], 16, dataset.num_classes)
model1 = GCN(sg1.ndata['feat'].shape[1], 16, dataset.num_classes)

In [46]:
train(cora_dgl,model)

In epoch 0, loss: 1.945, val acc: 0.096 (best 0.096), test acc: 0.106 (best 0.106)
In epoch 5, loss: 1.885, val acc: 0.111 (best 0.163), test acc: 0.134 (best 0.193)
In epoch 10, loss: 1.803, val acc: 0.251 (best 0.251), test acc: 0.277 (best 0.277)
In epoch 15, loss: 1.702, val acc: 0.440 (best 0.440), test acc: 0.475 (best 0.475)
In epoch 20, loss: 1.578, val acc: 0.584 (best 0.584), test acc: 0.605 (best 0.605)
In epoch 25, loss: 1.435, val acc: 0.662 (best 0.662), test acc: 0.683 (best 0.683)
In epoch 30, loss: 1.275, val acc: 0.693 (best 0.693), test acc: 0.702 (best 0.702)
In epoch 35, loss: 1.107, val acc: 0.706 (best 0.706), test acc: 0.717 (best 0.717)
In epoch 40, loss: 0.938, val acc: 0.725 (best 0.725), test acc: 0.728 (best 0.728)
In epoch 45, loss: 0.777, val acc: 0.743 (best 0.743), test acc: 0.743 (best 0.743)
In epoch 50, loss: 0.632, val acc: 0.763 (best 0.763), test acc: 0.755 (best 0.755)
In epoch 55, loss: 0.508, val acc: 0.769 (best 0.769), test acc: 0.758 (best 0

In [43]:
train(sg0, model0)

In epoch 0, loss: 1.945, val acc: 0.150 (best 0.150), test acc: 0.188 (best 0.188)
In epoch 5, loss: 1.810, val acc: 0.427 (best 0.427), test acc: 0.426 (best 0.426)
In epoch 10, loss: 1.652, val acc: 0.408 (best 0.427), test acc: 0.419 (best 0.426)
In epoch 15, loss: 1.500, val acc: 0.399 (best 0.427), test acc: 0.416 (best 0.426)
In epoch 20, loss: 1.359, val acc: 0.404 (best 0.427), test acc: 0.419 (best 0.426)
In epoch 25, loss: 1.221, val acc: 0.418 (best 0.427), test acc: 0.426 (best 0.426)
In epoch 30, loss: 1.085, val acc: 0.455 (best 0.455), test acc: 0.453 (best 0.453)
In epoch 35, loss: 0.951, val acc: 0.484 (best 0.484), test acc: 0.478 (best 0.478)
In epoch 40, loss: 0.822, val acc: 0.516 (best 0.516), test acc: 0.508 (best 0.508)
In epoch 45, loss: 0.703, val acc: 0.540 (best 0.540), test acc: 0.556 (best 0.556)
In epoch 50, loss: 0.594, val acc: 0.582 (best 0.582), test acc: 0.600 (best 0.600)
In epoch 55, loss: 0.497, val acc: 0.615 (best 0.615), test acc: 0.627 (best 0

In [44]:
train(sg0, model1)

In epoch 0, loss: 1.945, val acc: 0.178 (best 0.178), test acc: 0.188 (best 0.188)
In epoch 5, loss: 1.828, val acc: 0.263 (best 0.263), test acc: 0.259 (best 0.249)
In epoch 10, loss: 1.686, val acc: 0.329 (best 0.329), test acc: 0.307 (best 0.307)
In epoch 15, loss: 1.538, val acc: 0.390 (best 0.390), test acc: 0.384 (best 0.384)
In epoch 20, loss: 1.392, val acc: 0.432 (best 0.432), test acc: 0.432 (best 0.435)
In epoch 25, loss: 1.237, val acc: 0.455 (best 0.455), test acc: 0.465 (best 0.465)
In epoch 30, loss: 1.079, val acc: 0.535 (best 0.535), test acc: 0.506 (best 0.506)
In epoch 35, loss: 0.920, val acc: 0.577 (best 0.577), test acc: 0.568 (best 0.568)
In epoch 40, loss: 0.767, val acc: 0.606 (best 0.606), test acc: 0.597 (best 0.597)
In epoch 45, loss: 0.627, val acc: 0.638 (best 0.638), test acc: 0.625 (best 0.625)
In epoch 50, loss: 0.506, val acc: 0.667 (best 0.667), test acc: 0.670 (best 0.657)
In epoch 55, loss: 0.405, val acc: 0.681 (best 0.681), test acc: 0.684 (best 0